In [ ]:
%pip install opencv-python
%pip install tensorflow

In [4]:
import cv2
import numpy as np
import time
import tensorflow as tf

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Set up the webcam stream.
cap = cv2.VideoCapture(0)

# Set up the text display parameters.
font = cv2.FONT_HERSHEY_SIMPLEX
org = (50, 50)
fontScale = 1
color = (0, 255, 0)
thickness = 2

# Initialize the frame counter and FPS calculation.
frame_count = 0
start_time = time.time()

# Set up the video recording parameters.
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = None
recording = False
recording_start_time = 0
start_recording = False

# Initialize fire detection variables
consecutive_fire_frames = 0
consecutive_no_fire_frames = 0

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Preprocess the input image.
    resized = cv2.resize(frame, (256, 256), interpolation = cv2.INTER_AREA)
    normalized = resized.astype('float32') / 255.0
    input_data = np.expand_dims(normalized, axis=0)
    
    # Perform the inference.
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    # Get the predicted class and confidence.
    prediction = output_data[0][0]
    if prediction < 0.5:
        label = 'Fire'
        consecutive_fire_frames += 1
        consecutive_no_fire_frames = 0
        if consecutive_fire_frames == 20:
            print("Fire")
            consecutive_fire_frames = 0
            consecutive_no_fire_frames = 0
            start_recording = True
            recording_start_time = time.time()
            out = cv2.VideoWriter('fire1.mp4', fourcc, 20, (frame.shape[1], frame.shape[0],))
    else:
        label = 'No fire'
        consecutive_fire_frames = 0
        consecutive_no_fire_frames += 1
        if consecutive_no_fire_frames == 20:
            print("No fire")
            consecutive_fire_frames = 0
            consecutive_no_fire_frames = 0
            if recording:
                elapsed_time = time.time() - recording_start_time
                if elapsed_time > 10:
                    recording = False
                    start_recording = False
                    out.release()
                    out = None
    
    # Display the prediction and frame rate on the screen.
    cv2.putText(frame, label, org, font, fontScale, color, thickness, cv2.LINE_AA)
    frame_count += 1
    if frame_count % 10 == 0:
        end_time = time.time()
        fps = int(frame_count / (end_time - start_time))
        print('FPS:', fps)
        accuracy = round(prediction * 100, 2)
        print('Accuracy:', accuracy, '%')
        frame_count = 0
        start_time = end_time
    
    # Save the frames if fire is detected and recording is enabled.
    if start_recording:
        if time:
           if not recording:
            recording = True
           if out is not None:
            out.write(frame)
    
       # Display the resulting frame.
    cv2.imshow('Fire Detection', frame)

    # Exit if the 'q' key is pressed.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Release the video writer if it is still open.
if out is not None:
    out.release()

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

FPS: 14
Accuracy: 95.2 %
No fire
FPS: 30
Accuracy: 94.88 %
FPS: 31
Accuracy: 95.84 %
No fire
FPS: 28
Accuracy: 97.76 %
FPS: 31
Accuracy: 97.84 %
No fire
FPS: 29
Accuracy: 97.91 %
FPS: 31
Accuracy: 90.91 %
No fire
FPS: 28
Accuracy: 85.25 %
FPS: 29
Accuracy: 97.72 %
No fire
FPS: 31
Accuracy: 99.83 %
FPS: 26
Accuracy: 13.62 %
FPS: 33
Accuracy: 1.86 %
Fire
FPS: 30
Accuracy: 0.88 %
FPS: 30
Accuracy: 1.36 %
Fire
FPS: 27
Accuracy: 0.33 %
FPS: 30
Accuracy: 3.36 %
Fire
FPS: 30
Accuracy: 1.41 %
FPS: 32
Accuracy: 0.89 %
Fire
FPS: 26
Accuracy: 0.31 %
FPS: 30
Accuracy: 0.12 %
Fire
FPS: 31
Accuracy: 0.12 %
FPS: 30
Accuracy: 0.05 %
FPS: 29
Accuracy: 84.37 %
FPS: 30
Accuracy: 45.83 %
FPS: 29
Accuracy: 52.45 %
FPS: 30
Accuracy: 56.77 %
No fire
FPS: 29
Accuracy: 61.14 %
FPS: 30
Accuracy: 78.59 %
No fire
FPS: 30
Accuracy: 88.76 %
FPS: 30
Accuracy: 87.05 %
No fire
FPS: 29
Accuracy: 85.51 %
FPS: 30
Accuracy: 87.18 %
No fire
FPS: 30
Accuracy: 81.93 %
FPS: 27
Accuracy: 82.5 %
No fire
FPS: 32
Accuracy: 84.01 